In [3]:
import numpy as np

class ART1:
    def __init__(self, input_size, num_categories, vigilance):
        self.input_size = input_size
        self.num_categories = num_categories
        self.vigilance = vigilance

        # Weight initialization (set to 1s, classical ART1)
        self.weights = np.ones((num_categories, input_size * 2))  # for complement coding

    def complement_code(self, input_vector):
        # Add complement to input vector (1s and 0s assumed)
        return np.concatenate([input_vector, 1 - input_vector])

    def match_category(self, input_vector):
        # Calculate match (activation) as sum of minimums
        return np.array([np.sum(np.minimum(input_vector, w)) for w in self.weights])

    def vigilance_test(self, input_vector, category_index):
        # Match quality (fuzzy match) test
        numerator = np.sum(np.minimum(input_vector, self.weights[category_index]))
        denominator = np.sum(input_vector)
        return numerator / denominator >= self.vigilance

    def train(self, input_patterns):
        for pattern in input_patterns:
            input_vector = self.complement_code(pattern)

            while True:
                match_scores = self.match_category(input_vector)

                # Choose best matching category
                chosen = np.argmax(match_scores)

                # If it passes vigilance, learn
                if self.vigilance_test(input_vector, chosen):
                    self.weights[chosen] = np.minimum(self.weights[chosen], input_vector)
                    break
                else:
                    # Disable this category for current input
                    self.weights[chosen] = np.zeros_like(self.weights[chosen])

    def predict(self, input_vector):
        input_vector = self.complement_code(input_vector)
        match_scores = self.match_category(input_vector)
        return np.argmax(match_scores)

# --- Example usage ---
inputs = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 1],
    [1, 1, 0, 0],
    [0, 0, 1, 1]
])

art = ART1(input_size=4, num_categories=4, vigilance=0.7)
art.train(inputs)

print("\nPredictions:")
for input_vector in inputs:
    category = art.predict(input_vector)
    print(f"Input: {input_vector}, Predicted Category: {category}")



Predictions:
Input: [1 0 0 0], Predicted Category: 0
Input: [0 1 0 1], Predicted Category: 1
Input: [1 1 0 0], Predicted Category: 2
Input: [0 0 1 1], Predicted Category: 3


In [2]:
import numpy as np

class ART1:
    def __init__(self, input_size, num_categories, vigilance):
        self.input_size = input_size
        self.num_categories = num_categories
        self.vigilance = vigilance

        # Initialize weight matrix
        self.weights = np.ones((num_categories, input_size))

    def complement_code(self, input_vector):
        # Generate complement coded input
        return np.concatenate([input_vector, 1 - input_vector])

    def match_category(self, input_vector):
        # Compute matching scores
        scores = np.dot(self.weights, input_vector)
        norms = np.sum(input_vector)
        return scores / norms

    def train(self, inputs):
        for input_vector in inputs:
            input_vector = self.complement_code(input_vector)
            while True:
                match_scores = self.match_category(input_vector)
                chosen_category = np.argmax(match_scores)

                # Vigilance test
                if np.sum(np.minimum(input_vector, self.weights[chosen_category])) / np.sum(input_vector) >= self.vigilance:
                    # Update weights
                    self.weights[chosen_category] = np.minimum(input_vector, self.weights[chosen_category])
                    break
                else:
                    # Reset the category by setting its weight to zero
                    self.weights[chosen_category] = np.zeros_like(self.weights[chosen_category])

    def predict(self, input_vector):
        input_vector = self.complement_code(input_vector)
        match_scores = self.match_category(input_vector)
        return np.argmax(match_scores)

# Example usage
inputs = np.array([[1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1]])
art = ART1(input_size=6, num_categories=4, vigilance=0.7)
art.train(inputs)

print("Predictions:")
for input_vector in inputs:
    print(f"Input: {input_vector}, Predicted Category: {art.predict(input_vector)}")


Predictions:
Input: [1 0 0], Predicted Category: 0
Input: [0 1 0], Predicted Category: 1
Input: [1 1 0], Predicted Category: 2
Input: [0 0 1], Predicted Category: 3
